In [3]:
from pathlib import Path
import guitarpro
from tqdm import tqdm

In [4]:
DRIVE_PATH = Path('/Users/vlad/googledrive')

tabs_path = DRIVE_PATH / 'PlayMusic/tabs'
paths = list(tabs_path.glob('**/*.gp[3-5]'))
print(f'Found {len(paths)} files')

Found 49125 files


#### Parsing tabs

We would start with only bass tracks as they are simpler. Doing best to select bass tracks based on heuristics.

In [5]:
N_TOY = 50

In [6]:
from collections import defaultdict

N_FRETS = 24
N_STRINGS = 4  # base; 6 for standard guitar
INSTRUMENTS = range(32, 40)  # base; range(24, 31) for standard guitar

tracks_by_path = dict()

for path_num, path in enumerate(tqdm(paths)):
    # print(f'{path_num} parsing {path}')
    try:
        curl = guitarpro.parse(path)
    except guitarpro.GPException:
        print(f'   failed to parse {path}')
        continue
        
    tracks = []
    for track in curl.tracks:
        if all([
            track.settings.tablature, 
            len(track.strings) == N_STRINGS,
            track.fretCount == N_FRETS,
            track.channel.instrument in INSTRUMENTS,
        ]):
            tracks.append(track)
    # print(f'   found {len(guitar_tracks)}/{len(curl.tracks)} bass tracks')
    if tracks:
        tracks_by_path[path] = tracks
    if sum(map(len, tracks_by_path.values())) >= N_TOY:
        break

if len(tracks_by_path) == 0:
    raise Exception('No bass tracks found')

print(
    f'Found {len(tracks_by_path)}/{len(paths)} files with bass tracks, '
    f'total {sum(map(len, tracks_by_path.values()))} tracks'
)

  0%|          | 90/49125 [00:24<3:46:41,  3.61it/s]

Found 50/49125 files with bass tracks, total 50 tracks


#### Verifying tabs by printing one in a human-readable format

In [7]:
path = list(tracks_by_path.keys())[0]
track = tracks_by_path[path][0]

lines_by_voice = defaultdict(list)

for mi, measure in enumerate(track.measures):
    for vi, voice in enumerate(measure.voices):
        if vi != 0:
            assert not voice.beats  # only the first voice is usually non-empty
        if vi not in lines_by_voice:
            lines_by_voice[vi] = ['' for _ in range(N_STRINGS)]
        for bi, beat in enumerate(voice.beats):
            s = f'{beat.duration.value}{"·" if beat.duration.isDotted else ""}:'
            note_by_string = {note.string - 1: note for note in beat.notes}
            for si in range(N_STRINGS):
                lines_by_voice[vi][si] += s
                if si in note_by_string:
                    lines_by_voice[vi][si] += f'{note_by_string[si].value:<3}'
                else:
                    lines_by_voice[vi][si] += f'{"-":<3}'
                lines_by_voice[vi][si] += ' '
        for si in range(N_STRINGS):
            lines_by_voice[vi][si] += '|'

lines = lines_by_voice[0]  # only the first voice is non-empty, ignoring others
screen_width = 80
print(path)
print(track.name)
for screen_num in range(0, len(lines[0]), screen_width):
    for line in lines:
        print(line[screen_num:screen_num+screen_width])
    print()

/Users/vlad/googledrive/PlayMusic/tabs/White Lion/White Lion - Lady Of The Valley.gp4
Bass
1:-   |4·:7   4·:7   4:7   |8:0   4·:7   4:7   4:7   |4·:7   4·:7   4:7   |8:0  
1:-   |4·:-   4·:-   4:-   |8:-   4·:-   4:-   4:-   |4·:-   4·:-   4:-   |8:-  
1:-   |4·:-   4·:-   4:-   |8:-   4·:-   4:-   4:-   |4·:-   4·:-   4:-   |8:-  
1:-   |4·:-   4·:-   4:-   |8:-   4·:-   4:-   4:-   |4·:-   4·:-   4:-   |8:-  

 4·:7   4:7   4:7   |4·:7   4·:7   4:7   |8:0   4·:7   4:7   4:7   |4·:7   4·:7 
 4·:-   4:-   4:-   |4·:-   4·:-   4:-   |8:-   4·:-   4:-   4:-   |4·:-   4·:- 
 4·:-   4:-   4:-   |4·:-   4·:-   4:-   |8:-   4·:-   4:-   4:-   |4·:-   4·:- 
 4·:-   4:-   4:-   |4·:-   4·:-   4:-   |8:-   4·:-   4:-   4:-   |4·:-   4·:- 

  4:7   |8:0   4·:7   8:7   8·:-   8·:-   |4·:-   2:-   8:-   |4·:-   4:-   8·:-
  4:-   |8:-   4·:-   8:-   8·:-   8·:-   |4·:-   2:0   8:0   |4·:-   4:-   8·:-
  4:-   |8:-   4·:-   8:-   8·:-   8·:-   |4·:-   2:-   8:-   |4·:-   4:-   8·:-
  4:-   |8:-   4

#### Encoding into flat string
 
Now attempting to encode multi-string tabs into a flat string, suitable for training a transformer. In an NLP world, our token would encode all notes that sound at the same time during one beat, along with the beat duration. We would separate beats with a space (like words of text), and separate measures with a "|" character (like paragraphs/sentences).

In [24]:
import math

duration_alphabet = ['1', '2', '4', '8', 'F', 'H', 'G', 'I']
strings_alphabet = ['E', 'A', 'D', 'G', 'B', 'e']

def encode_track(track):
    out = []
    
    for mi, measure in enumerate(track.measures):
        beats = measure.voices[0].beats
        if not any(len(b.notes) > 0 for b in beats):
            continue
        
        for bi, beat in enumerate(beats):
            s = ''
            s += duration_alphabet[int(math.log2(beat.duration.value))]
            if beat.duration.isDotted:
                s += '.'
            
            if beat.notes:
                notes = []
                for n in beat.notes:
                    if n.type == guitarpro.NoteType.tie:
                        v = '~'
                    elif n.type == guitarpro.NoteType.dead:
                        v = 'x'
                    elif n.type == guitarpro.NoteType.rest:
                        print(f'Rest node in {path}, track {track.name}, measure {mi}, beat {bi}')
                        v = 'r'
                    else:
                        v = n.value
                    notes.append(f'{strings_alphabet[n.string - 1]}{v}')
                s += "".join(notes)
            out.append(s)
        out.append('|')
    return out

encoded = encode_track(track)
print(path)
print(' '.join(encoded))

/Users/vlad/googledrive/PlayMusic/tabs/White Lion/White Lion - Lady Of The Valley.gp4
4.E7 4.E7 4E7 | 8E~ 4.E7 4E7 4E7 | 4.E7 4.E7 4E7 | 8E~ 4.E7 4E7 4E7 | 4.E7 4.E7 4E7 | 8E~ 4.E7 4E7 4E7 | 4.E7 4.E7 4E7 | 8E~ 4.E7 8E7 8.G1 8.G0 | 4. 2A0 8A~ | 4. 4 8.G1 8.G0 | 4D3 8D3 2E5 8E~ | 4.E6 4A7 8.G1 8.G0 | 4. 2A0 8A~ | 4. 4 8.G1 8.G0 | 4D3 8D3 2E5 8E~ | 4.E6 4A7 8.G1 8.G0 | 4. 2A0 8A~ | 4. 4 8.G1 8.G0 | 4D3 8D3 4.E5 4E5 | 4.E6 4A7 8.G1 8.G0 | 4. 2A0 8A~ | 4 8 4 8.G1 8.G0 | 4D3 8D3 4.E5 4E5 | 4.E6 4.A7 4A7 | 4.D8 4D7 8 4D8 | 8D~ 4D7 8 4D8 8E0 8E0 | 1E~ | 1E~ | 2. 8 8D0 | 1A0 | 2.A~ 4 | 2. 8 8D0 | 4.A0 8D0 4A0 4A0 | 2.D3 8D~ 8D3 | 4.D3 8D3 8A0 8D3 8 8G3 | 2.G1 8G~ 8 | 4.G1 8 8G1 8G3 8D0 8A0 | 2.D3 8D~ 8G3 | 8E2 8E0 8A3 8A2 8A0 8D3 8D0 8G3 | 8D0 8D0 8D0 8D0 8D0 8D0 8G3 8D0 | 4D~ 8D0 8D0 8D0 8D0 8D0 8D0 | 8D1 8D1 8D1 8D1 8D1 8D1 8D1 8D3 | 8D~ 8D3 8D3 8D3 8D3 8D1 8D0 8G3 | 8D0 8D0 8D0 8D0 8D0 8D0 8G3 8D0 | 8D~ 8D0 8D0 8D0 8D0 8D0 8D0 8D0 | 8D1 8D1 8D1 8D1 8D1 8D1 8D1 8D3 | 4D~ 2. | 2.D3 8D~ 8G3 | 

#### Decode into human-readable tab

Decoding back into human-readable tab to verify correctness.

In [9]:
def decode_and_print(encoded: list[str]):
    decoded_by_string = defaultdict(str)

    for measure in ' '.join(encoded).strip().split('|'):
        for beat in measure.strip().split(' '):
            if not beat:
                continue

            i = 0
            duration = str(2 ** duration_alphabet.index(beat[i]))
            i += 1
            
            if i < len(beat) and beat[i] == '·':
                duration += '·'
                i += 1
    
            for si in range(N_STRINGS):
                decoded_by_string[si] += f'{duration}:'
             
            note_by_string = {}
            cur_string, cur_note = None, ''
            while i < len(beat):
                if beat[i] in strings_alphabet:
                    if cur_string is not None:
                        note_by_string[cur_string] = cur_note
                    cur_string = strings_alphabet.index(beat[i])
                    cur_note = ''
                else:
                    cur_note += beat[i]
                i += 1
            
            if cur_string is not None:
                note_by_string[cur_string] = cur_note
            
            for si in range(N_STRINGS):
                if si in note_by_string:
                    decoded_by_string[si] += f'{note_by_string[si]:<3}'
                else:
                    decoded_by_string[si] += f'{"-":<3}'
                decoded_by_string[si] += ' '
    
        for si in range(N_STRINGS):
            decoded_by_string[si] += '|'
    
    screen_width = 80
    for screen_num in range(0, len(decoded_by_string[0]), screen_width):
        for line in decoded_by_string.values():
            print(line[screen_num:screen_num+screen_width])
        print()
        
decode_and_print(encoded)

4·:7   4·:7   4:7   |8:~   4·:7   4:7   4:7   |4·:7   4·:7   4:7   |8:~   4·:7  
4·:-   4·:-   4:-   |8:-   4·:-   4:-   4:-   |4·:-   4·:-   4:-   |8:-   4·:-  
4·:-   4·:-   4:-   |8:-   4·:-   4:-   4:-   |4·:-   4·:-   4:-   |8:-   4·:-  
4·:-   4·:-   4:-   |8:-   4·:-   4:-   4:-   |4·:-   4·:-   4:-   |8:-   4·:-  

 4:7   4:7   |4·:7   4·:7   4:7   |8:~   4·:7   4:7   4:7   |4·:7   4·:7   4:7  
 4:-   4:-   |4·:-   4·:-   4:-   |8:-   4·:-   4:-   4:-   |4·:-   4·:-   4:-  
 4:-   4:-   |4·:-   4·:-   4:-   |8:-   4·:-   4:-   4:-   |4·:-   4·:-   4:-  
 4:-   4:-   |4·:-   4·:-   4:-   |8:-   4·:-   4:-   4:-   |4·:-   4·:-   4:-  

 |8:~   4·:7   8:7   8·:-   8·:-   |4·:-   2:-   8:-   |4·:-   4:-   8·:-   8·:-
 |8:-   4·:-   8:-   8·:-   8·:-   |4·:-   2:0   8:~   |4·:-   4:-   8·:-   8·:-
 |8:-   4·:-   8:-   8·:-   8·:-   |4·:-   2:-   8:-   |4·:-   4:-   8·:-   8·:-
 |8:-   4·:-   8:-   8·:1   8·:0   |4·:-   2:-   8:-   |4·:-   4:-   8·:1   8·:0

   |4:-   8:-   2:5   8:~

In [10]:
print(encoded[:9])

['4·E7', '4·E7', '4E7', '|', '8E~', '4·E7', '4E7', '4E7', '|']


In [11]:
import torch
torch.random.manual_seed(42)
torch.cuda.random.manual_seed(42)

#### Tokenizing


In [38]:
import pprint
import itertools
from torch.utils.data import random_split, TensorDataset
from tokenizers.implementations import SentencePieceBPETokenizer

print(f'Total tracks: {len(tracks_by_path)}')
tracks = list(itertools.chain.from_iterable(tracks_by_path.values()))

tokenizer = SentencePieceBPETokenizer()
vocab_size = 50_000
lines = [' '.join(encode_track(t)) for t in tracks]
tokenizer.train_from_iterator(lines, vocab_size=vocab_size, special_tokens=['<s>', '<unk>'])
print(f'Tokenizer vocab ({tokenizer.get_vocab_size()} tokens):')
pprint.pprint(sorted(tokenizer.get_vocab().items(), key=lambda kv: kv[1]))

Total tracks: 50



Tokenizer vocab (352 tokens):
[('<s>', 0),
 ('<unk>', 1),
 ('.', 2),
 ('0', 3),
 ('1', 4),
 ('2', 5),
 ('3', 6),
 ('4', 7),
 ('5', 8),
 ('6', 9),
 ('7', 10),
 ('8', 11),
 ('9', 12),
 ('A', 13),
 ('D', 14),
 ('E', 15),
 ('F', 16),
 ('G', 17),
 ('H', 18),
 ('x', 19),
 ('|', 20),
 ('~', 21),
 ('▁', 22),
 ('▁8', 23),
 ('G0', 24),
 ('▁F', 25),
 ('▁8G0', 26),
 ('▁8G', 27),
 ('▁FG0', 28),
 ('▁|', 29),
 ('▁8D', 30),
 ('▁4', 31),
 ('▁FG', 32),
 ('▁FD', 33),
 ('▁8G1', 34),
 ('▁8G3', 35),
 ('▁2', 36),
 ('▁8D3', 37),
 ('▁8G5', 38),
 ('▁4G', 39),
 ('▁4D', 40),
 ('▁8G8', 41),
 ('▁8D1', 42),
 ('▁8D5', 43),
 ('▁8D0', 44),
 ('▁FA', 45),
 ('▁8.', 46),
 ('▁8A', 47),
 ('▁FG1', 48),
 ('▁1', 49),
 ('▁FG5', 50),
 ('▁4.', 51),
 ('▁4G0', 52),
 ('▁FG3', 53),
 ('▁8G~', 54),
 ('▁8G10', 55),
 ('▁2D', 56),
 ('▁8G2', 57),
 ('▁FG8', 58),
 ('▁8G4', 59),
 ('▁FG~', 60),
 ('▁FD1', 61),
 ('▁2G', 62),
 ('▁4D3', 63),
 ('▁8D2', 64),
 ('▁FG2', 65),
 ('▁FA0', 66),
 ('▁8.G', 67),
 ('▁FGx', 68),
 ('▁FD3', 69)

In [43]:
print(tokenizer.encode(' '.join(encode_track(track))).ids)

[162, 162, 169, 29, 158, 162, 169, 169, 29, 162, 162, 169, 29, 158, 162, 169, 169, 29, 162, 162, 169, 29, 158, 162, 169, 169, 29, 162, 162, 169, 29, 158, 162, 179, 95, 102, 29, 51, 167, 153, 29, 51, 31, 95, 102, 29, 63, 37, 140, 158, 29, 224, 219, 95, 102, 29, 51, 167, 153, 29, 51, 31, 95, 102, 29, 63, 37, 140, 158, 29, 224, 219, 95, 102, 29, 51, 167, 153, 29, 51, 31, 95, 102, 29, 63, 37, 247, 212, 29, 224, 219, 95, 102, 29, 51, 167, 153, 29, 31, 23, 31, 95, 102, 29, 63, 37, 247, 212, 29, 224, 276, 219, 29, 88, 11, 136, 23, 204, 29, 92, 136, 23, 204, 151, 151, 29, 328, 29, 328, 29, 73, 23, 44, 29, 291, 29, 242, 31, 29, 73, 23, 44, 29, 345, 44, 106, 106, 29, 172, 92, 37, 29, 128, 37, 114, 37, 23, 35, 29, 272, 54, 23, 29, 166, 23, 34, 35, 44, 114, 29, 172, 92, 35, 29, 184, 151, 81, 101, 114, 37, 44, 35, 29, 44, 44, 44, 44, 44, 44, 35, 44, 29, 98, 44, 44, 44, 44, 44, 44, 29, 42, 42, 42, 42, 42, 42, 42, 37, 29, 92, 37, 37, 37, 37, 42, 44, 35, 29, 44, 44, 44, 44, 44, 44, 35, 44, 29, 92, 44,

In [52]:
from transformers import GPT2LMHeadModel, GPT2Config

config = GPT2Config(
    bos_token_id=0,
    eos_token_id=0,
    vocab_size=tokenizer.get_vocab_size(),
    n_positions=120,  # 1024
    n_embd=96,        # 768,
    n_layer=6,        # 12,
    n_head=6,         # 12,
)

model = GPT2LMHeadModel(config)
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(352, 96)
    (wpe): Embedding(120, 96)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0

In [80]:
class Dataset(TensorDataset):
    def __init__(self, tracks: list[guitarpro.Track], n_ctx: int):
        self.n_ctx = n_ctx
        
        data = torch.LongTensor([0])
        for track in tracks:
            tokens = tokenizer.encode(' '.join(encode_track(track))).ids + [0]
            data = torch.cat([data, torch.LongTensor(tokens)])
            
        super().__init__(data)
        test_n = min(1000, int(len(self) * 0.1))
        self.train, self.test = random_split(self, [len(self) - test_n, test_n])

    def __getitem__(self, idx: int):
        t = torch.LongTensor(self.tensors[0][idx:idx + self.n_ctx])
        return {"input_ids": t, "labels": t}

ds = Dataset(tracks, n_ctx=config.n_positions)
print(f'Dataset size: {len(ds)}')
print(f'Train size: {len(ds.train)}')
print(f'Test size: {len(ds.test)}')

Dataset size: 39482
Train size: 38482
Test size: 1000


In [71]:
def sample(num_seqs=2, max_length=10):
    torch.manual_seed(42)
    torch.cuda.manual_seed_all(42)
    for i, seq in enumerate(model.generate(
        max_length=max_length,
        top_p=0.95,
        num_return_sequences=num_seqs,
        do_sample=True, 
        top_k=50,
        pad_token_id=0,
        eos_token_id=0,
        bos_token_id=0,
    )):
        seq = tokenizer.decode(seq.tolist())
        print(i + 1, seq)
        decode_and_print(seq.split())

sample()

Generate config GenerationConfig {
  "bos_token_id": 0,
  "eos_token_id": 0,
  "transformers_version": "4.26.1"
}

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


1 
|
|
|
|

2 17 8D1 8E13 F. 8.D0 4.E7 2.D1 1D~13
1:-   8:-   8:13  16:-   8:-   4:7   2:-   1:-   |
1:-   8:-   8:-   16:-   8:-   4:-   2:-   1:-   |
1:-   8:1   8:-   16:-   8:0   4:-   2:1   1:~13 |
1:-   8:-   8:-   16:-   8:-   4:-   2:-   1:-   |



In [79]:
import os
from transformers import Trainer, TrainingArguments, TrainerCallback
from transformers.trainer_utils import get_last_checkpoint

save_dir = DRIVE_PATH / "AI" / "gtp_gpt"
save_dir.mkdir(exist_ok=True)
if last_checkpoint_dir := get_last_checkpoint(str(save_dir)):
    last_checkpoint_dir = Path(last_checkpoint_dir)
    print([t.name for t in last_checkpoint_dir.iterdir()])

class MyCallback(TrainerCallback):
    def on_save(self, args, state, control, **kwargs):
        sample()

trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir=str(save_dir),
        report_to=['wandb'] if 'WANDB_API_KEY' in os.environ else None,
        evaluation_strategy="epoch",
        overwrite_output_dir=True,
        eval_steps=500,
        save_steps=500,
        save_total_limit=2,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        ignore_data_skip=True,
    ),
    train_dataset=ds.train,
    callbacks=[MyCallback],
)
# trainer.train(resume_from_checkpoint=last_checkpoint_dir)
trainer.train()


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 38482
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 57723
  Number of trainable parameters = 716544


tensor([ 55,  55,  55,  55, 173,  55,  89,  29,  41,  41,  41,  41,  41,  41,
         41,  41,  29,  55,  55,  55,  55,  55,  55,  55,  55,  29,  50,  80,
         50,  53,  50,  28,  76,  26,  26,  28,  28,  29,  26,  28,  28,  28,
         95,  28,  28,  28,  28,  28,  95,  29,  50,  80,  50,  53,  50,  28,
         76,  26,  26,  28,  28,  29,  26,  28,  28,  28,  95,  28,  28,  25,
         28,  28,  28,  23,  29,  50,  80,  50,  53,  50,  28,  76,  26,  26,
         28,  28,  29,  26,  28,  28,  28,  95,  28,  28,  28,  28,  28,  95,
         29,  50,  80,  50,  53,  50,  28,  76,  26,  26,  28,  28,  29,  26,
         28,  28,  28,  95,  28,  28,  25,  28])
tensor([106,  29, 172,  29,  37,  23,  36,  29,  37,  63,  63,  37,  29,  63,
        129, 106,  29, 142,  29,  26,  23,  36,  29,  26,  52,  52,  26,  29,
         52,  52, 106,  29, 172,  29,  37,  23,  36,  29,  37,  63,  63,  37,
         29,  63, 129, 106,  29, 142,  29,  26,  23,  36,  29,  26,  52,  52,
         26,  2

Epoch,Training Loss,Validation Loss


tensor([ 98,  98,  29, 135,  84,  84,  54,  35,  29,  84,  84,  84,  54,  37,
         29,  98,  98,  98,  98,  29,  98,  98,  98,  98,  29, 135,  84,  84,
         54,  35,  29,  84,  84,  84,  84,  29,  23,  23,  23,  23,  41, 164,
         29,  75,  38,  26,  35,  38,  26,  41,  26,  29,  54,  26,  26,  23,
         26,  26,  23,  26,  29,  75,  38,  26,  35,  26,  26,  75,  38,  29,
         26,  35,  38,  26,  41,  26,  26,  26,  29,  75,  38,  26,  35,  38,
         26,  41,  26,  29,  54,  26,  52,  36,  29,  75,  38,  26,  35,  26,
         26,  75,  38,  29,  26,  35,  38,  26,  41,  26,  26,  26,  29,  75,
         38,  26,  35,  38,  26,  41,  26,  29])
tensor([ 28,  28,  29,  28,  28,  28,  28,  60,  28,  28,  28, 109, 109, 109,
         28,  80,  80,  80,  80,  29,  28,  28,  28,  28,  60,  28,  28,  28,
         28,  28,  28,  28,  60,  28,  28,  28,  29,  28,  28,  28,  28,  60,
         28,  28,  28,  70,  70,  70,  70,  86,  86,  86,  86,  29,  28,  28,
         28,  2

KeyboardInterrupt: 